In [1]:
from bs4 import BeautifulSoup
import requests

In [39]:
url='https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE'
r = requests.get(url)
print(url, r.status_code)
soup = BeautifulSoup(r.content,'lxml')

https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE 200


In [41]:
soup

<!DOCTYPE html>
<html itemscope="" itemtype="http://schema.org/WebPage" lang="en">
<head>
<meta charset="utf-8"/>
<script>
    window.dataLayer = [
        {
            "user": {
                "loginStatus": "logged out",
                "id": "",
                "personal": {
                    "language": ""
                }
            },
                                                "screen": {
                "name": "search results list",
                "language": "en"
            }

        }
    ];
</script>
<!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=false;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-M7L459B');</script>
<!-- End Google Tag Manager -->
<!-- Special case : hardcode Kameleoon script -->

In [ ]:
link_list_district = []
for elem in soup.find_all('a'):
    link_list_district.append(elem.get('href'))
    print(elem.get('href'))